# Models Exploration using CUB dataset

## References
* [Transfer Learning with Hub](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub)
* [`tf.keras.utils.image_dataset_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory)
* [Limiting GPU Memory Growth](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth)

## Setup

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import datetime
import os
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub
from keras.utils.layer_utils import count_params


### Limit GPU memory allocation

In [2]:
def limit_memory_growth(limit=True):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, limit)
            logical_gpus = tf.config.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

---

## Enumerate Datasets to test

In [3]:
import pathlib

flowers_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
flowers_data_dir = tf.keras.utils.get_file('flower_photos', origin=flowers_dataset_url, untar=True)
flowers_data_dir = pathlib.Path(flowers_data_dir)

datasets = [
    ('CUB-200-2011', '/mnt/cub/CUB_200_2011/images'),
    ('flowers', flowers_data_dir),
]

## Dataset

In [4]:
# build a dataset object from a directory of images
def build_dataset(
    dataset,
    image_size,
    preprocess_input = None,
    batch_size = 64,
):
   
    train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
        dataset[1],
        batch_size = batch_size,
        validation_split = 0.2,
        image_size = image_size,
        subset = "both",
        shuffle = True, # default but here for clarity
        seed=42,
        label_mode="categorical" # enables one-hot encoding (use 'int' for sparse_categorical_crossentropy loss)
    )
    
    # Retrieve class names
    # (can't do this after converting to PrefetchDataset?)
    class_names = train_ds.class_names
    
    # Prefetch images
    train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    
    # apply preprocessing function
    train_ds = train_ds.map(
        lambda x, y: (preprocess_input(x), y),
        num_parallel_calls = 16,
    )
    val_ds = val_ds.map(
        lambda x, y: (preprocess_input(x), y),
        num_parallel_calls = 16,
    )
    
    return (train_ds, val_ds, class_names)

---

## Enumerate Models to test

In [5]:
# Array of tuples describing the models to be tested
# in the form: (model_handle, input_image_size, preprocessing_function)
# where the model_handle is a model building function or a url to a tfhub feature model
base_models_metadata = [
    (
        'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4',
        224,
        # https://www.tensorflow.org/hub/common_signatures/images#input
        # The inputs pixel values are scaled between -1 and 1, sample-wise.
        tf.keras.applications.mobilenet_v2.preprocess_input,
    ),
    (
        'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4',
        299,
        # The inputs pixel values are scaled between -1 and 1, sample-wise.
        tf.keras.applications.inception_v3.preprocess_input,
    ),
    (
        'https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/5',
        299,
        # The inputs pixel values are scaled between -1 and 1, sample-wise.
        tf.keras.applications.inception_v3.preprocess_input,
    ),
    (
        tf.keras.applications.Xception,
        299,
        # The inputs pixel values are scaled between -1 and 1, sample-wise.
        tf.keras.applications.xception.preprocess_input,
    ),
    (
        tf.keras.applications.resnet.ResNet101,
        224,
        tf.keras.applications.resnet50.preprocess_input,
    ),
    (
        tf.keras.applications.ResNet50,
        224,
        tf.keras.applications.resnet50.preprocess_input,
    ),
    (
        tf.keras.applications.InceptionResNetV2,
        299,
        tf.keras.applications.inception_resnet_v2.preprocess_input,
    ),
    (
        tf.keras.applications.efficientnet_v2.EfficientNetV2B0,
        224,
        # The preprocessing logic has been included in the EfficientNetV2
        # model implementation. Users are no longer required to call this
        # method to normalize the input data. This method does nothing and
        # only kept as a placeholder to align the API surface between old
        # and new version of model.
        tf.keras.applications.efficientnet_v2.preprocess_input,
    )
]

In [6]:
# return a name that accurately describes the model building function or
# the tfhub model (by url) that was passed
def get_model_name( model_handle ):
    
    if callable(model_handle):
        return f'keras.applications/{model_handle.__name__}'
    else:
        split = model_handle.split('/')
        return f'tfhub/{split[-5]}.{split[-4]}.{split[-3]}'
    

## Model Building

In [7]:

class FullModel(tf.keras.Sequential):
    
    def __init__(
        self,
        base_model_metadata,
        num_classes,
        dropout,
        thawed_base_model_layers,
        data_augmentation = False,
    ):
        super().__init__(name = "full_model")
        
        # Get base_model_information
        model_handle, input_dimension, preprocess_input = base_model_metadata
        
        # TODO: only really needed on training data...?
        #  - should be a function of the dataset, no?
        if data_augmentation:
            self.add(self.build_data_augmentation())
        
        self.base_model = self.build_base_model(
            model_handle,
            thawed_base_model_layers = thawed_base_model_layers,
        )
        self.add(self.base_model)

        self.classifier_model = self.build_classifier_model(
            num_classes,
            dropout,
        )
        self.add(self.classifier_model)
    
    @staticmethod
    def build_data_augmentation():
        data_augmentation = keras.Sequential([
            layers.RandomFlip(
                "horizontal",
            ),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
        ], name = "data_augmentation")

        return data_augmentation

    @staticmethod
    def build_base_model(
        base_model_handle,
        thawed_base_model_layers = 0,
        name="base_model",
    ):
        # If model_handle is a model building function, use that function
        if callable(base_model_handle):
            base_model = model_handle(
                include_top=False,
                weights='imagenet',
                pooling = 'avg',
            )

        # otherwise build a layer from the tfhub url that was passed as a string
        else:
            base_model = hub.KerasLayer(
                base_model_handle,
                name=name,
            )

        # Print Base model weights
        print("\nBase Model:")
        FullModel.print_weight_counts(base_model)

        return base_model
    
    # Freeze base model?
    def freeze_base_model(
        self,
        thawed_base_model_layers = 0,
    ):
        if thawed_base_model_layers == 0:
            self.base_model.trainable = False
        elif thawed_base_model_layers > 0:
            for layer in self.base_model.layers[:(-1*thawed_base_model_layers)]:
                layer.trainable = False
        else:
            self.base_model.trainable = True
    
    @staticmethod
    def build_classifier_model(
        num_classes,
        dropout,
    ):
        model = Sequential()
        model.add(
            layers.Dense(
                num_classes,
                # activation = 'softmax',
            )
        )

        model.add(
            layers.Dropout(dropout)
        )

        model.add(
            layers.Activation("softmax", dtype="float32")
        )
        
        return model

    # Print model weight counts
    @staticmethod
    def print_weight_counts(model):
        print(f'Non-trainable weights: {count_params(model.non_trainable_weights)}')
        print(f'Trainable weights: {count_params(model.trainable_weights)}')

---

## Run Results Logging

In [8]:
class RunLogging():

    hdf_key = "runs"

    def __init__(
        self,
        data_dir = "./",
        hdf_filename = "runs.h5",
    ):
        self.data_dir = data_dir
        self.filename =  os.path.join( data_dir, hdf_filename )
        self.df = self.load_metadata()

    def load_metadata(self):
        if ( os.path.exists(self.filename) ):
            return pd.read_hdf( self.filename, self.hdf_key )
        else: return pd.DataFrame()

    def save_df(self):
        self.df.to_hdf(self.filename, self.hdf_key)

    def add_run(self, params, log_dir, time, scores):

        cols = {
            **params,
            'time': time,
            'log_dir': log_dir,
            'scores.loss': scores[0],
            'scores.accuracy': scores[1],
            'scores.top3': scores[2],
            'scores.top10': scores[3],
        }

        new_run = pd.DataFrame([cols])

        self.df = pd.concat([self.df, new_run])
        self.save_df()

---

## Build and run all models

* Note regarding `thawed_base_model_layers` and full model architecture ([reference](https://stackoverflow.com/questions/64227483/what-is-the-right-way-to-gradually-unfreeze-layers-in-neural-network-while-learn))
![image](https://i.stack.imgur.com/JLJqv.png)
* [Another great reference](https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751)

In [9]:
# Hyperparameters

## Dataset Hyperparameters
dataset = datasets[0]
batch_size = 64

## Full Model Hyperparameters

### Input Model Hyperparameters
data_augmentation = False

### Base Model Hyperparameters
thawed_base_model_layers = 0 # base_model completely frozen
# thawed_base_model_layers = n # last 'n' layers of base_model unfrozen
# thawed_base_model_layers = -1 # all base_model layers unfrozen

### Classifier Model Hyperparameters
classifier_dropout = 0.33

## Training Hyperparameters
max_epochs = 2

### Optimizer Function Hyperparameters
learning_rate = 0.0005

### Loss function hyperparameters
label_smoothing = 0.2

# use logits?

# vary classifier architecture?

In [10]:
class Run():
    
    def __init__(
        self,
        base_model_metadata,
    ):
        model_handle, input_dimension, preprocess_input = base_model_metadata
        self.image_size = (input_dimension, input_dimension)

        

In [ ]:
def run_model(
    base_model_metadata,
):
    model_handle, input_dimension, preprocess_input = base_model_metadata

    image_size = (input_dimension, input_dimension)
    
    # Build dataset/pipeline
    train_ds, val_ds, class_names = build_dataset(
        dataset,
        batch_size = batch_size,
        image_size = image_size,
        preprocess_input = preprocess_input,
    )
    
    # Build model
    model = FullModel(
        base_model_metadata,
        len(class_names),
        classifier_dropout,
        thawed_base_model_layers = thawed_base_model_layers,
        data_augmentation = data_augmentation,
    )
    
    # Compile model
    # Sparse vs non-sparse CCE https://www.kaggle.com/general/197993
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate),
        # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        loss=tf.keras.losses.CategoricalCrossentropy(
            # from_logits=True,
            label_smoothing = label_smoothing,
        ),
        metrics=[
            'accuracy',
            tf.keras.metrics.CategoricalAccuracy(),
            tf.keras.metrics.CategoricalCrossentropy(),
            tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="Top3"),
            tf.keras.metrics.TopKCategoricalAccuracy(k=10, name="Top10"),
        ],
    )

    # Logging
    base_model_id = get_model_name(model_handle)
    run_log_dir = os.path.join( run_logs.data_dir, dataset[0], base_model_id )
    print(f'\n{run_log_dir}')
    
    # Tensorboard logs
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = run_log_dir,
        histogram_freq = 1,
    )
    
    # Early stopping
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        # monitor='val_sparse_categorical_accuracy',
        monitor = 'val_loss',
        patience = 5,
        min_delta = 0.01,
    )
    
    # Model Checkpoints for saving best model weights
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        os.path.join(run_log_dir, 'best_model' ),
        save_best_only = True,
        monitor = 'val_loss',
        # mode = 'min', # should be chosen correctly based on monitor value
    )
    
    best_batch = (0, float('inf'))
    
    # Train
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=max_epochs,
        callbacks=[
            tensorboard_callback,
            early_stopping_callback,
            model_checkpoint_callback,
            tf.keras.callbacks.LambdaCallback(
                on_epoch_end = lambda batch, logs: print((batch, logs['val_loss']) if logs['val_loss'] < best_batch[1] else best_batch),
            )
        ]
    )
    
    print(history)
    print(max(history.history['val_loss']))
    
    history_df = pd.DataFrame(history.history)

In [11]:
# Run Logging
run_logs = RunLogging(
    data_dir = f'models_cub_02_logs',
)

# for each base model
for base_model_metadata in base_models_metadata:
    run_model(base_model_metadata)
    
    
    
    


Found 11788 files belonging to 200 classes.
Using 9431 files for training.
Using 2357 files for validation.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089

Base Model:
Non-trainable weights: 2257984
Trainable weights: 0

models_cub_02_logs/CUB-200-2011/tfhub/google.tf2-preview.mobilenet_v2
Epoch 1/2
148/148 [==============================] - ETA: 0s - loss: 5.1552 - accuracy: 0.0579 - categorical_accuracy: 0.0579 - categorical_crossentropy: 4.9883 - Top3: 0.1233 - Top10: 0.2486INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1280, 200), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f178412b9a0>, 139739386151600), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1280, 200), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f178412b9a0>, 139739386151600), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(200,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1779f14790>, 139739384855920), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(200,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f1779f14790>, 139739384855920), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(1280, 200), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f178412b9a0>, 139739386151600), {}).


...

INFO:tensorflow:Assets written to: models_cub_02_logs/CUB-200-2011/keras.applications/EfficientNetV2B0/best_model/assets


(0, 3.2268424034118652)
148/148 [==============================] - 68s 320ms/step - loss: 4.4660 - accuracy: 0.2149 - categorical_accuracy: 0.2149 - categorical_crossentropy: 4.1237 - Top3: 0.3316 - Top10: 0.4606 - val_loss: 3.2268 - val_accuracy: 0.5397 - val_categorical_accuracy: 0.5397 - val_categorical_crossentropy: 2.5459 - val_Top3: 0.7650 - val_Top10: 0.9270
Epoch 2/2
148/148 [==============================] - ETA: 0s - loss: 3.4334 - accuracy: 0.5103 - categorical_accuracy: 0.5103 - categorical_crossentropy: 2.6496 - Top3: 0.6137 - Top10: 0.7176

INFO:tensorflow:Assets written to: models_cub_02_logs/CUB-200-2011/keras.applications/EfficientNetV2B0/best_model/assets


INFO:tensorflow:Assets written to: models_cub_02_logs/CUB-200-2011/keras.applications/EfficientNetV2B0/best_model/assets


(1, 2.8062069416046143)
148/148 [==============================] - 42s 288ms/step - loss: 3.4334 - accuracy: 0.5103 - categorical_accuracy: 0.5103 - categorical_crossentropy: 2.6496 - Top3: 0.6137 - Top10: 0.7176 - val_loss: 2.8062 - val_accuracy: 0.6759 - val_categorical_accuracy: 0.6759 - val_categorical_crossentropy: 1.9971 - val_Top3: 0.8693 - val_Top10: 0.9555
3.2268424034118652


In [12]:
tf.config.threading.get_inter_op_parallelism_threads()

0